In [3]:
from sklearn.feature_extraction.text import *
from sklearn.svm import SVC
import json
import numpy as np
import nltk.corpus
import nltk

In [ ]:
stop_words = nltk.corpus.stopwords.words('english') + list(string.punctuation)
def clean_article(s):
    return [i for i in nltk.word_tokenize(s) if i not in stop_words]

def load_data():
    data = json.load(open('stories.json'))
    for dataset in ['train', 'test', 'dev']:
        for idx, article in enumerate(data[dataset]):
            data[dataset][idx]['Content'] = clean_article(article['Content'])
            #data[dataset][idx]['Regions'] = article['Regions']
            #data[dataset][idx]['Persons / Groups'] = article['Persons / Groups']
            #data[dataset][idx]['Places'] = article['Places']
    return data

In [ ]:
# load training dataset from CoNll2003
train_content = np.genfromtxt('train_c.txt', dtype=str,usecols=(0, 3))

train_data=[]
label=[]
i=-1

for f in train_content:
    if f[0] == '-DOCSTART-':
        train_data.append([])
        #label.append([])
        i+=1
    if f[0] != '-DOCSTART-':
        train_data[i].append(f[0])
        label.append(f[1])

# load test dataset
data =load_data()
corpus = [' '.join(article['Content']) for article in data['test']]

In [ ]:
# combine the two dataset
data=[]
y=[]
for p in range(len(train_data)):
    data.append(' '.join(train_data[p])) 
    y.append(label[p]) 
for a in range(len(corpus)): 
    data.append(corpus[a])

# vectorize the whole text data
vectorizer = TfidfVectorizer(sublinear_tf=True, norm='l2')
txt_vec = vectorizer.fit_transform(data)

In [ ]:
# create the feature vector for each word
x = []
#y_updated =[]
for j in range(len(data)):
    words_lst=nltk.word_tokenize(data[j])
    for i in range(len(words_lst)):
        length = (2*window_sz+1)*txt_vec.shape[1]
        feature = np.zeros(length)
        
        if i>= 2 and i<len(words_lst)-2:
            if vectorizer.vocabulary_.get(words_lst[i].lower()) != None:
                idx = vectorizer.vocabulary_.get(words_lst[i].lower())
                feature[idx] = txt_vec[j,idx]
            
            if vectorizer.vocabulary_.get(words_lst[i-1].lower()) != None:
                idx_pre1 = vectorizer.vocabulary_.get((words_lst[i-1]).lower())
                feature[idx_pre1+txt_vec.shape[1]*1] = txt_vec[j, idx_pre1]
                
            if  vectorizer.vocabulary_.get(words_lst[i-2].lower()) != None:  
                idx_pre2 = vectorizer.vocabulary_.get((words_lst[i-2]).lower())
                feature[idx_pre2+txt_vec.shape[1]*2] = txt_vec[j, idx_pre2]
                
            if vectorizer.vocabulary_.get(words_lst[i+1].lower()) != None: 
                idx_aft1 = vectorizer.vocabulary_.get((words_lst[i+1]).lower())
                feature[idx_aft1+txt_vec.shape[1]*3] = txt_vec[j, idx_aft1]
            
            if vectorizer.vocabulary_.get(words_lst[i+2].lower()) != None:
                idx_aft2 = vectorizer.vocabulary_.get((words_lst[i+2]).lower())
                feature[idx_aft2+txt_vec.shape[1]*4] = txt_vec[j, idx_aft2]
            
            x.append(feature)
            
        elif i<2:
            
            if vectorizer.vocabulary_.get(words_lst[i].lower()) != None:
                idx = vectorizer.vocabulary_.get(words_lst[i].lower())
                feature[idx] = txt_vec[j,idx]
            
            if vectorizer.vocabulary_.get(words_lst[i+1].lower()) != None: 
                idx_aft1 = vectorizer.vocabulary_.get((words_lst[i+1]).lower())
                feature[idx_aft1+txt_vec.shape[1]*3] = txt_vec[j, idx_aft1]
            
            if vectorizer.vocabulary_.get(words_lst[i+2].lower()) != None:
                idx_aft2 = vectorizer.vocabulary_.get((words_lst[i+2]).lower())
                feature[idx_aft2+txt_vec.shape[1]*4] = txt_vec[j, idx_aft2]
            
            x.append(feature)
        
        else:
            
            if vectorizer.vocabulary_.get(words_lst[i].lower()) != None:
                idx = vectorizer.vocabulary_.get(words_lst[i].lower())
                feature[idx] = txt_vec[j,idx]
            
            if vectorizer.vocabulary_.get(words_lst[i-1].lower()) != None:
                idx_pre1 = vectorizer.vocabulary_.get((words_lst[i-1]).lower())
                feature[idx_pre1+txt_vec.shape[1]*1] = txt_vec[j, idx_pre1]
                
            if  vectorizer.vocabulary_.get(words_lst[i-2].lower()) != None:  
                idx_pre2 = vectorizer.vocabulary_.get((words_lst[i-2]).lower())
                feature[idx_pre2+txt_vec.shape[1]*2] = txt_vec[j, idx_pre2]
                
            x.append(feature)

x_feature=np.asarray(x)

In [ ]:
# fit
#classes = {'B-ORG', 'I-ORG', 'B-MISC','I-MISC','B-PER','I-PER','B-LOC', 'I-LOC', 'O'}
svm_classifier = SVC(kernel='poly', degree=2)
y=np.asarray(label)
svm_classifier.fit(x_feature, y)
X_test=x_feature[0:100]
pre = svm_classifier.predict(X_test)